In [14]:
import sys
sys.path.append('../..')

In [15]:
from src.recommenders.baseline_recommender import BaselineRecommender
from src.evaluation import average_precision

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [17]:
books_df = pd.read_csv('../../data/Books.csv', delimiter=';')
ratings_df = pd.read_csv('../../data/Ratings.csv', delimiter=';')
users_df = pd.read_csv('../../data/Users.csv', delimiter=';')

/var/folders/yn/16crnc792zg2yr78zf0qg0j40000gq/T/ipykernel_28441/2786065433.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv('../../data/Users.csv', delimiter=';')


In [18]:
train_ratings, test_ratings = train_test_split(ratings_df, test_size=0.3, random_state=42)

In [19]:
model = BaselineRecommender()
model.fit(items=books_df, users=users_df, ratings=train_ratings)

In [20]:
test_users = test_ratings['User-ID'].unique()

In [21]:
best_items = model.predict(test_users)

In [24]:
best_items[0]

['0439139597',
 '0439064864',
 '0439136350',
 '0590353403',
 '043935806X',
 '0812550706',
 '0553375407',
 '0439136369',
 '0440219078',
 '0439139600']

In [ ]:
len(test_users)

53334

In [ ]:
from tqdm import tqdm

In [ ]:

rating_mask = test_ratings['Rating'] >= 5

mean_ap = 0
for i in tqdm(range(len(test_users))):
    user_mask = test_ratings['User-ID'] == test_users[i]
    relevant_items = test_ratings.loc[(user_mask & rating_mask)].sort_values(by='Rating', ascending=False)
    if len(relevant_items) == 0:
        continue
    ap = average_precision(best_items[i], relevant_items['ISBN'].to_numpy())
    mean_ap += ap

mean_ap /= len(test_users)
mean_ap

  0%|          | 0/53334 [00:00<?, ?it/s]

100%|██████████| 53334/53334 [00:49<00:00, 1082.37it/s]


0.000632279760813595